In [1]:
# Description: Predict the price of ETH using ML Model

In [2]:
pip install yfinance --upgrade --no-cache-dir

     |████████████████████████████████| 6.3MB 7.5MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=d58de0f7d0eff25fc9370811780c813c7f0b1012b8eb16c51ad1e00a0e447576
  Stored in directory: /tmp/pip-ephem-wheel-cache-iom9jygc/wheels/fe/a0/79/b73d4a0c535b421b88fc7b393936b371fabbfeaf979eca4050
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
# Importing libraries
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as web
import datetime
import math
import random
from datetime import date
plt.style.use('fivethirtyeight')

In [4]:
from google.colab import files
uploaded = files.upload()

Saving NIFTYNEXT50-Symbols.csv to NIFTYNEXT50-Symbols.csv


In [5]:
# For SP500
"""sp500 = pd.read_csv('S&P500-Symbols.csv').to_numpy()
sp500 = sp500[:, 1]
print(sp500)"""

# FOr NIFTY50
"""nifty50 = pd.read_csv('NIFTY50-Symbols.csv').to_numpy()
nifty50 = nifty50[:, 1]
print(nifty50)"""

# FOr NIFTY50
niftynext50 = pd.read_csv('NIFTYNEXT50-Symbols.csv').to_numpy()
niftynext50 = niftynext50[:, 1]
print(niftynext50)

['ABBOTINDIA' 'ACC' 'ADANIENT' 'ADANIGREEN' 'ADANITRANS' 'ALKEM'
 'AMBUJACEM' 'APOLLOHOSP' 'AUROPHARMA' 'DMART' 'BAJAJHLDNG' 'BANDHANBNK'
 'BERGEPAINT' 'BIOCON' 'BOSCHLTD' 'CADILAHC' 'COLPAL' 'DABUR' 'DLF' 'GAIL'
 'GODREJCP' 'HAVELLS' 'HDFCAMC' 'HINDPETRO' 'ICICIGI' 'ICICIPRULI' 'IGL'
 'INDUSTOWER' 'NAUKRI' 'INDIGO' 'JUBLFOOD' 'LTI' 'LUPIN' 'MARICO'
 'MOTHERSUMI' 'MRF' 'MUTHOOTFIN' 'NMDC' 'PETRONET' 'PIDILITIND' 'PEL'
 'PGHH' 'PNB' 'SBICARD' 'SIEMENS' 'TORNTPHARM' 'UBL' 'MCDOWELL-N' 'VEDL'
 'YESBANK']


In [6]:
# Create new list of stocks
companies = ['GRASIM.NS', 'TITAN.NS', 'KOTAKBANK.NS', 'TATACONSUM.NS', 'HDFCLIFE.NS', 'NTPC.NS', 'HINDALCO.NS', 'ITC.NS', 'BAJAJFINSV.NS', 'BAJAJ-AUTO.NS', 'ULTRACEMCO.NS', 'BAJFINANCE.NS',	'NESTLEIND.NS','BHARTIARTL.NS','BRITANNIA.NS', 'SHREECEM.NS',	'HEROMOTOCO.NS', 'MARUTI.NS', 'CIPLA.NS', 'TATASTEEL.NS', 'ONGC.NS', 'ICICIBANK.NS', 'LT.NS', 'COALINDIA.NS', 'RELIANCE.NS', 'WIPRO.NS', 'TCS.NS', 'TECHM.NS', 'IIFLWAM.NS', 'TATASTLBSL.NS', 'NH.NS', 'OIL.BO', 'DEEPAKNI.BO', 'AMBER.NS', 'PHOENIXLTD.BO', 'ROUTE.NS', 'PGHL.NS', 'GRANULES.BO', 'FORTIS.BO', 'RATNAMANI.NS', 'NBCC.NS','BIRLACORPN.NS', 'ERIS.NS', 'INTELLECT.NS', 'WABCOINDIA.BO', 'WELSPUNIND.NS', 'CGCL.NS', 'BBTC.NS', 'SOLARINDS.BO', 'CYIENT.NS', 'MCX.BO', 'IIFL.NS', 'KAJARIACER.BO']

In [7]:
# Define Bullish Harami Pattern Test
def bullish_harami(yest_o, yest_c, tod_o, tod_c):
  if yest_o > yest_c and tod_o < tod_c and yest_o > tod_c and yest_c < tod_o:
    return True
  else:
    return False

In [8]:
# Define Bearish Engulfing Candle Test
def bearish_engulfing(yest_o, yest_c, tod_o, tod_c):
  if yest_o > yest_c and tod_o < tod_c and tod_o < yest_c and tod_c > 0.5*(yest_o + yest_c):
    return True
  else:
    return False

In [9]:
# Import Data and calculate returns for 1 year trading
def check_return(i, stock):
  yf.pdr_override()
  start_date = (datetime.datetime.now() - datetime.timedelta(days = ((i + 1) * 365))).strftime("%Y-%m-%d")
  end_date = (datetime.datetime.now() - datetime.timedelta(days = (i * 365))).strftime("%Y-%m-%d")
  
  try:
    # For NIFTYNEXT50 stock is like ABBOTINDIA but we need it as ABBOTINDIA.NS
    stock = stock + '.NS'
    print(stock)
    df = web.get_data_yahoo(stock, data_source='yahoo', start=start_date, end=end_date)
    date_flag = True
  except:
    date_flag = False
  if date_flag:
    df.dropna(how = 'any', inplace = True)
    df['tod_o'] = df['Open']
    df['tod_c'] = df['Close']
    df['yest_o'] = df['Open'].shift(1)
    df['yest_c'] = df['Close'].shift(1)

    # Add ATR 20 Value to df
    df['max_diff'] = pd.Series()
    df['h_l'] = df['High'] - df['Low']
    df['h_c'] = (df['High'] - df['Close']).abs()
    df['l_c'] = (df['Low'] - df['Close']).abs()
    df['max_diff'] = df[['h_l', 'h_c', 'l_c']].max(axis = 1)
    df['ATR20'] = df['max_diff'].rolling(window = 20).mean()
    df['ATR20'] = df['ATR20'].shift(1)
    df = df.iloc[20:,[0, 1, 6, 7, 8, 9, 14]]

    df['buy'] = pd.Series()
    df['sell'] = pd.Series()
    df['funds'] = np.nan

    last_pos = 0
    buy_pos = 0
    last_funds = 100000.0

    harami_flag = False
    bear_engulf_flag = False
    stop_loss = np.nan
    target_price = np.nan
    pos_hit = 0.0
    neg_hit = 0.0

    for index, row in df.iterrows():
      if (harami_flag or bear_engulf_flag) and last_pos == 0:
        row['buy'] = row['tod_o']
        row['sell'] = np.nan
        row['funds'] = last_funds
        last_pos = int(last_funds / row['tod_o'])
        target_price = row['tod_o'] + 3 * row['ATR20']
        stop_loss = row['tod_o'] - 3 * row['ATR20']
      elif last_pos > 0 and row['yest_c'] > target_price:
        row['buy'] = np.nan
        row['sell'] = row['tod_o']
        last_funds = last_pos * row['tod_o']
        row['funds'] = last_funds
        last_pos = 0
        pos_hit += 1
      elif last_pos > 0 and row['yest_c'] < stop_loss:
        row['buy'] = np.nan
        row['sell'] = row['tod_o']
        last_funds = last_pos * row['tod_o']
        row['funds'] = last_funds
        last_pos = 0
        neg_hit += 1
      elif last_pos > 0:
        row['buy'] = np.nan
        row['sell'] = np.nan
        last_funds = last_pos * row['tod_c']
        row['funds'] = last_funds
      else:
        row['buy'] = np.nan
        row['sell'] = np.nan
        row['funds'] = last_funds

      harami_flag = bullish_harami(row['yest_o'],	row['yest_c'], row['tod_o'],	row['tod_c'])
      bear_engulf_flag = bearish_engulfing(row['yest_o'],	row['yest_c'], row['tod_o'],	row['tod_c'])
    
    # hit_rate of strategy i.e. (no. of time Target Price reached) / (no. of times Stop Loss reached)
    try:
      hit_rate = (pos_hit/(pos_hit + neg_hit))
    except ZeroDivisionError:
      hit_rate = np.nan
    # annual return on investment in percent rounded to 2 decimals
    try:
      roi = round((np.array(df['funds'])[-1] * 0.001) - 100, 2)
    except IndexError:
      roi = 'SomeError'

    # use (stock[:-3]) for indian stocks and (stock) for US because they don't have .BO or .NS in the end
    ret_arr = [str(stock), df['funds'].max(), df['funds'].min(), start_date,  hit_rate, roi]
    ret_df = pd.DataFrame([ret_arr])
    
    ret_df.to_csv('niftynext50_strategy_returns.csv', mode='a', header=False)

    return str(stock) + str(i)
  else:
    return 'Date Error'

In [10]:
# Do calculations and download images for each stock in our nifty database
for c in niftynext50:
  for i in range(10):
    print(check_return(i, c))

ABBOTINDIA.NS
[*********************100%***********************]  1 of 1 completed
ABBOTINDIA.NS0
ABBOTINDIA.NS


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


[*********************100%***********************]  1 of 1 completed
ABBOTINDIA.NS1
ABBOTINDIA.NS
[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


ABBOTINDIA.NS2
ABBOTINDIA.NS


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


[*********************100%***********************]  1 of 1 completed
YESBANK.NS8
YESBANK.NS


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


[*********************100%***********************]  1 of 1 completed
YESBANK.NS9


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [11]:
df = pd.read_csv('niftynext50_strategy_returns.csv')
df

,0,ABBOTINDIA.NS,100000.0,47693.8505859375,2020-07-16,0.5,-47.94
0,0,ABBOTINDIA.NS,129582.000000,87908.496094,2019-07-17,0.857143,-8.73
1,0,ABBOTINDIA.NS,103120.195312,68462.000000,2018-07-17,0.400000,-29.77
2,0,ABBOTINDIA.NS,136382.946289,93749.151123,2017-07-17,1.000000,27.82
3,0,ABBOTINDIA.NS,105933.297852,88495.000000,2016-07-17,0.500000,0.14
4,0,ABBOTINDIA.NS,120215.000000,79071.301758,2015-07-18,0.333333,-15.67
...,...,...,...,...,...,...,...
494,0,YESBANK.NS,166974.717407,95138.401794,2015-07-18,1.000000,66.97
495,0,YESBANK.NS,158320.500000,93550.798950,2014-07-18,1.000000,47.92
496,0,YESBANK.NS,244156.196297,100000.000000,2013-07-18,0.714286,122.46
497,0,YESBANK.NS,159279.121826,99350.821838,2012-07-18,0.666667,20.88
